In [133]:
from ipynb.fs.full.Chromosome import Chromosome
from ipynb.fs.full.Individual import Individual
from ipynb.fs.full.Population import Population, tournamentSelection
from ipynb.fs.full.Crossovers import singlePointCrossoverObj, twoPointsCrossoverObj, threePointsCrossoverObj, probUniformCrossoverObj

from ipywidgets import Text, ToggleButton, ToggleButtons, FloatSlider, HBox, Label, VBox
from ipywidgets import IntSlider, Layout, Button, AppLayout, IntProgress, Box, Textarea

import time
import matplotlib.pyplot as plt
import pandas as pd

In [60]:
def initButton(desc, color, width, height):
    return ToggleButton(description = desc, disabled = False, button_style = color, 
                        layout = Layout(width = width, height = height))

def initHbox(text, maximum, step, val):
    return HBox([
        Label(text, layout = Layout(width = '22%')), 
        IntSlider(min = 0, max = maximum, step = step, value = val, layout = Layout(width = '50%'))])


def initFloatHbox(text, maximum, step, val):
    return HBox([
        Label(text, layout = Layout(width = '22%')), 
        FloatSlider(min = 0, max = maximum, step = step, value = val, layout = Layout(width = '50%'))])

#### Konfiguracja

In [61]:
def initConfPanel():
    global populationSl, bitsSl, epochSl
    confHeader = initButton('Konfiguracja', 'danger', '72%', '50px')
    populationSl = initHbox('Liczba osobników populacji', 200, 1, 20)
    bitsSl = initHbox('Dokłandność chromosomów', 20, 1, 8)
    epochSl = initHbox('Liczba epok', 500, 1, 100)

    confSliders = Box(
        children = [populationSl, bitsSl, epochSl], 
        layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
    )
    return AppLayout(header = confHeader, center = confSliders)

#### Operatory genetyczne

In [62]:
def initGenPanel():
    global selectionBtn, crossoverBtn, mutationBtn, selectionSl, crossoverSl, mutationSl
    
    genHeader = initButton('Operatory genetyczne', 'danger', '72%', '50px')

    selectionBtn = ToggleButtons(
        options = ['najlepszych', 'kołem ruletki', 'turniejowa', 'strategia elitarna'],
        description = 'Selekcja: ', button_style = 'danger')

    crossoverBtn = ToggleButtons(
        options = ['jednopunktowe', 'dwupunktowe', 'trzypunktowe', 'jednorodne'],
        description = 'Krzyżowanie: ', button_style = 'warning')

    mutationBtn = ToggleButtons(
        options = ['jednopunktowa', 'dwupunktowa', 'brzegowa'],
        description = 'Mutacja: ', button_style = 'success')

    selectionSl = initHbox('Procent osobników', 100, 1, 30) 
    crossoverSl = initFloatHbox('Prawdopodobieństwo krzyżowania', 1, 0.1, 0.9)
    mutationSl = initFloatHbox('Prawdopodobieństwo mutacji', 1, 0.1, 0.2)

    buttons = Box( children = [selectionBtn, crossoverBtn, mutationBtn], 
        layout = Layout(display = 'flex', flex_flow = 'column')
    )
    geneticSliders = Box(
        children = [selectionSl, crossoverSl, mutationSl], 
        layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
    )
    selectionBtn.observe(selectionBtnClicked, 'value')
    box = Box(children = [buttons, geneticSliders], layout = Layout(display = 'flex', flex_flow = 'column'))
    
    return AppLayout(header = genHeader, center = box)

#### Event listeners

In [63]:
def updateSelectionSlider(minimum, text):
    selectionSl.layout.display = 'block'
    selectionSl.layout.display = 'flex'
    selectionSl.layout.flex_flow = 'row'
    selectionSl.children[1].min = minimum
    selectionSl.children[0].value = text

def selectionBtnClicked(change):
    if change['new'] == 'najlepszych' or change['new'] == 'strategia elitarna':
        updateSelectionSlider(0, 'Procent osobników')
    if change['new'] == 'kołem ruletki':
        selectionSl.layout.display = 'none'
    if change['new'] == 'turniejowa':
        updateSelectionSlider(1, 'Wielkość turnieju')
    
def incrementEpochProgress():
    epochProgress.value += 1

def startApp(change):
    popSize = populationSl.children[1].value
    chromoPrec = bitsSl.children[1].value
    epoch = epochSl.children[1].value
    selection = selectionBtn.value
    crossover = crossoverBtn.value
    mutation = mutationBtn.value
    if selection != 'kołem ruletki':
        selectionParam = selectionSl.children[1].value
    else: 
        selectionParam = 0
    crossoverProb = crossoverSl.children[1].value
    mutationProb = mutationSl.children[1].value  
    
    app.children[0].layout.display = 'none'
    app.children[1].layout = Layout()
    epochProgress.max = epoch
    
    updateConfInfo(popSize, chromoPrec, epoch, selection, selectionParam, 
                   crossover, crossoverProb, mutation, mutationProb)
    
    # to delete !!!
    for i in range(50):
        time.sleep(0.1)
        incrementEpochProgress()

#### Panel konfiguracji parametrów

In [64]:
def initFirstPanel():
    global startBtn
    
    configLayout = initConfPanel()
    geneticLayout = initGenPanel()
    startBtn = initButton('Rozpocznij', 'success', '72%', '50px')
    startBtn.observe(startApp, 'value')
    
    return Box(
        children = [configLayout, geneticLayout, startBtn], 
        layout = Layout(display = 'flex', flex_flow = 'column'))

#### Panel progresu epok

In [65]:
def initProgressPanel():
    global epochProgress
    epochProgress = IntProgress(
        value = 0, min = 0, max = 10, step = 1, description = 'Epoka:', 
        bar_style = '', layout = Layout(display = 'none'))
    return epochProgress

def updateTime(value):
    timeOutput.children[1].value = value

#### Panel wyników: czas obliczeń, wykresy, zapis do pliku

In [66]:
def showPlot(data, ylabel):
    plt.figure(figsize = (20, 30))
    plt.plot(data, color = 'red')
    plt.xlabel('Epoka')
    plt.ylabel(ylabel)
    plt.show()

def showFirstPlot(value):
    if value['new'] == True:
        showPlot(getDataPlot1(), 'Wykres zależności wartości funkcji celu od kolejnej iteracji') # to change
    boothFuncBtn.value = False

def showSecondPlot(value):
    if value['new'] == True:
        showPlot(getDataPlot2(), 'Wykres zależności średniej wartości funkcji celu od kolejnej iteracji') # to change
    meanBoothFuncBtn.value = False
    
def showThreePlot(value):
    if value['new'] == True:
        showPlot(getDataPlot3(), 'Wykres zależności odchylenia standardowego od kolejnej iteracji') # to change
    stdBtn.value = False
    
global dataPlot1

def setDataPlot1(newData):
    global dataPlot1
    dataPlot1 = newData

def getDataPlot1():
    global dataPlot1
    return dataPlot1 if 'dataPlot1' in globals() else []

global dataPlot2

def setDataPlot2(newData):
    global dataPlot2
    dataPlot2 = newData

def getDataPlot2():
    global dataPlot2
    return dataPlot2 if 'dataPlot2' in globals() else []

global dataPlot3

def setDataPlot3(newData):
    global dataPlot3
    dataPlot3 = newData

def getDataPlot3():
    global dataPlot3
    return dataPlot3 if 'dataPlot3' in globals() else []

In [67]:
def restartApp(change):
    app.children[2].layout.display = 'none'
    app.children[0].layout =  Layout(display = 'flex', flex_flow = 'column')
    epochProgress.value = 0

def ifFinished(change):
    if change['new'] == epochProgress.max:
        app.children[1].layout.display = 'none'
        app.children[2].layout = Layout(display = 'flex', flex_flow = 'row')

In [73]:
def saveFile(value):
    if value['new'] == True:
        results = { 
            'wartości funkcji celu' : getDataPlot1(),
            'średnie wartości funkcji celu' : getDataPlot2(),
            'odchylenie standardowe' : getDataPlot3(),
        } 
        path = fileInput.children[1].value  
        df = pd.DataFrame(results)
        try:
            df.to_csv(path)
        except Exception as f:
            f.write('an error message')
    fileButton.value = False

In [171]:
def initResultPanel():
    global infoDesc, resultsArea, boothFuncBtn, meanBoothFuncBtn, stdBtn, fileInput, fileButton, returnBtn
    
    params = initButton('Wybrane parametry', '', '100%', '50px')
    infoDesc = Textarea(
        value = '', disabled = True, 
        layout = Layout(width = "480px", height = "250px")
    )
    
    resultsHeader = initButton('Uzyskane rezultaty', '', '100%', '50px')
    resultsArea = Textarea(
        value = '', disabled = True, 
        layout = Layout(width = "480px", height = "50px")
    )

    boothFuncBtn = initButton('Wykres zależności wartości funkcji celu od kolejnej iteracji', 'danger', '100%', '') 
    meanBoothFuncBtn = initButton('Wykres zależności średniej wartości funkcji celu od kolejnej iteracji', 'danger', '100%', '')
    stdBtn = initButton('Wykres zależności odchylenia standardowego od kolejnej iteracji', 'danger', '100%', '') 
    plotsBox = Box(children = [boothFuncBtn, meanBoothFuncBtn, stdBtn], 
        layout = Layout(display = 'flex', flex_flow = 'column', margin = "10px"))

    fileInput = HBox([Label('Zapis do pliku:', layout = Layout(width = '180px')), 
        Text(value = '', placeholder = 'lokalizacja pliku', disabled = False, layout = Layout(width = '80%cd'))])
    fileButton = initButton('Zapisz', 'warning', '20%', '')
    fileBox = Box(children = [fileInput, fileButton], 
              layout = Layout(display = 'flex', flex_flow = 'row', margin = "10px"))

    returnBtn = initButton('Skonfiguruj ustawienia algorytmu ponownie', 'success', '100%', '')
    
    left = VBox(children = [params, infoDesc])
    right = VBox(children = [resultsHeader, resultsArea, plotsBox, fileBox, returnBtn])

    boothFuncBtn.observe(showFirstPlot, 'value')
    meanBoothFuncBtn.observe(showSecondPlot, 'value')
    stdBtn.observe(showThreePlot, 'value')
    returnBtn.observe(restartApp, 'value')
    fileButton.observe(saveFile, 'value')
    epochProgress.observe(ifFinished, 'value')

    return Box([left, right], layout = Layout(display = 'none'))

In [166]:
def updateConfInfo(popSize, prec, epoch, selection, selParam, crossover, crProb, mutation, mutProb):
    out = 'Liczba osobników populacji: ' + str(popSize) + \
        '\nDokładność reprezentacji chromosomów: ' + str(prec) + \
        '\nLiczba epok: ' + str(epoch) + \
        '\nSelekcja: ' + selection
    if selection == 'najlepszych' or selection == 'strategia elitarna':
        out += '\nProcent osobników: ' + str(selParam)
    if selection == 'turniejowa':
        out += '\nWielkość turnieju: ' + str(selParam)    
    out += '\nKrzyżowanie: ' + crossover + \
        '\nPrawdopodobieństwo krzyżowania: ' + str(crProb) + \
        '\nMutacja: ' + mutation + \
        '\nPrawdopodobieństwo krzyżowania: ' + str(mutProb)

    infoDesc.value = out
    
def updateResults(time, timeUnit, result):
    resultsArea.value = 'Czas wykonania: ' + str(time) + ' ' + timeUnit + '\nWynik: ' + str(result)

In [85]:
def initApp(): 
    global app
    app = Box(
        children = [initFirstPanel(), initProgressPanel(), initResultPanel()], 
        layout = Layout(display = 'flex', flex_flow = 'column'))
    return app